In [9]:
import pandas as pd

In [ ]:
data = {
    'Categoria': ['A', 'B', 'A', 'B', 'A'],
    'Producto': ['Producto1', 'Producto2', 'Producto3', 'Producto4', 'Producto5'],
    'Ventas': [100, 200, 150, 300, 50],
    'Cantidad': [5, 10, 8, 15, 2]
} 